In [8]:
from transformers import BertModel
from data_dir import pretrained_3kmer_dir
from utils.data_generator import _data_generator_mtl
from multitask_learning import init_model_mtl
import os
dataloader = _data_generator_mtl()
model = init_model_mtl(pretrained_3kmer_dir, head="bert", config=os.path.join('models', 'config', 'mtl.json'))
for step, batch in enumerate(dataloader):
    input_ids, attn_mask, label_prom, label_ss, label_polya = tuple(t for t in batch)
    output = model(input_ids, attn_mask)
    # print(output.keys())
    bert = model.shared_layer
    bert_output = bert(input_ids, attn_mask)[0]
    print(bert_output.shape)
    print(output['prom'].shape)



torch.Size([1, 512, 768])
torch.Size([1, 512, 2])


In [6]:
print(bert_model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(69, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
 

In [3]:
from transformers import BertForMaskedLM
from data_dir import pretrained_3kmer_dir
model = BertForMaskedLM.from_pretrained(pretrained_3kmer_dir)
bert_layer = model.bert
sum(p.numel() for p in bert_layer.parameters() if p.requires_grad)

85503744

In [ ]:
mtl_seq = [
    ["ATGC" * 128, 0, 0, 1],
    ["TGCG" * 128, 1, 0, 0],
    ["GACT" * 128, 0, 1, 0],
    ["CACG" * 128, 0, 0, 0],
    ["CCAT" * 128, 0, 0, 0],
]
import pandas as pd
import os
df = pd.DataFrame(mtl_seq, columns=["sequence", "label_prom", "label_ss", "label_polya"])

In [4]:
from random import randint
import os
import pandas as pd
sequences = ['ATGC' * 128, 'TGAC' * 128, 'GATC' * 128, "AGCC" * 128, "TGGA" * 128]
labels = [''.join(['E' if randint(0, 255) % 2 == 0 else '.' for i in range(len(s))]) for s in sequences]

seq2seq_datasample_csv = os.path.join("sample", "seq2seq", "sample.csv")
os.makedirs(os.path.dirname(seq2seq_datasample_csv), exist_ok=True)
if os.path.exists(seq2seq_datasample_csv):
    os.remove(seq2seq_datasample_csv)

df = pd.DataFrame(
    [[seq, label] for seq, label in zip(sequences, labels)],
    columns=['sequence', 'label']
)
df.to_csv(seq2seq_datasample_csv)

